In [15]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [16]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nicolas Bravo,MX,18.4500,-88.9167,88.68,47,24,3.06,few clouds
1,1,Felanitx,ES,39.4696,3.1483,68.95,70,40,7.61,scattered clouds
2,2,Kinablangan,PH,7.6947,126.5503,81.27,71,100,5.35,overcast clouds
3,3,Dunedin,NZ,-45.8742,170.5036,52.81,89,83,8.99,heavy intensity rain
4,4,Lompoc,US,34.6391,-120.4579,82.13,61,0,1.99,clear sky


In [17]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 70
What is your desired maximum temperature for your trip? 100


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nicolas Bravo,MX,18.4500,-88.9167,88.68,47,24,3.06,few clouds
2,2,Kinablangan,PH,7.6947,126.5503,81.27,71,100,5.35,overcast clouds
4,4,Lompoc,US,34.6391,-120.4579,82.13,61,0,1.99,clear sky
10,10,Presidencia Roque Saenz Pena,AR,-26.7852,-60.4388,80.96,61,85,5.95,overcast clouds
11,11,Rikitea,PF,-23.1203,-134.9692,74.64,81,100,18.86,light rain
16,16,Puerto Penasco,MX,31.3167,-113.5333,78.91,31,0,7.72,clear sky
21,21,Kalmunai,LK,7.4167,81.8167,81.55,78,100,2.77,overcast clouds
25,25,Ribeira Grande,PT,38.5167,-28.7000,72.00,94,75,12.66,broken clouds
30,30,Nouadhibou,MR,20.9310,-17.0347,76.98,61,0,18.41,clear sky
32,32,Cururupu,BR,-1.8283,-44.8683,86.16,62,89,8.39,overcast clouds


In [19]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nicolas Bravo,MX,18.4500,-88.9167,88.68,47,24,3.06,few clouds
2,2,Kinablangan,PH,7.6947,126.5503,81.27,71,100,5.35,overcast clouds
4,4,Lompoc,US,34.6391,-120.4579,82.13,61,0,1.99,clear sky
10,10,Presidencia Roque Saenz Pena,AR,-26.7852,-60.4388,80.96,61,85,5.95,overcast clouds
11,11,Rikitea,PF,-23.1203,-134.9692,74.64,81,100,18.86,light rain
...,...,...,...,...,...,...,...,...,...,...
717,717,Pandan,PH,14.0453,124.1694,79.75,87,93,10.25,overcast clouds
719,719,Palaikastron,GR,35.2000,26.2500,73.74,73,0,7.02,clear sky
722,722,Ipixuna,BR,-1.7625,-48.8043,75.15,73,57,5.12,broken clouds
723,723,Singaparna,ID,-7.3515,108.1110,70.56,95,89,0.92,overcast clouds


In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Nicolas Bravo,MX,88.68,few clouds,18.4500,-88.9167,
2,Kinablangan,PH,81.27,overcast clouds,7.6947,126.5503,
4,Lompoc,US,82.13,clear sky,34.6391,-120.4579,
10,Presidencia Roque Saenz Pena,AR,80.96,overcast clouds,-26.7852,-60.4388,
11,Rikitea,PF,74.64,light rain,-23.1203,-134.9692,
16,Puerto Penasco,MX,78.91,clear sky,31.3167,-113.5333,
21,Kalmunai,LK,81.55,overcast clouds,7.4167,81.8167,
25,Ribeira Grande,PT,72.00,broken clouds,38.5167,-28.7000,
30,Nouadhibou,MR,76.98,clear sky,20.9310,-17.0347,
32,Cururupu,BR,86.16,overcast clouds,-1.8283,-44.8683,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.


In [24]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Kinablangan,PH,81.27,overcast clouds,7.6947,126.5503,Danny Moreno Property
4,Lompoc,US,82.13,clear sky,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
10,Presidencia Roque Saenz Pena,AR,80.96,overcast clouds,-26.7852,-60.4388,Hotel Gualok
11,Rikitea,PF,74.64,light rain,-23.1203,-134.9692,People ThankYou
16,Puerto Penasco,MX,78.91,clear sky,31.3167,-113.5333,Playa Bonita Resort


In [25]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [27]:
len(clean_hotel_df)

329

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))